In [1]:
from pathlib import Path

import os
import sys

current_path = Path(os.getcwd())
sys.path.append(str(current_path.parent))
os.environ['VECTORIAN_CPP_IMPORT'] = "1"

In [2]:
import vectorian

In [3]:
import spacy
#nlp = spacy.load("en_core_web_trf")
nlp = spacy.load("en_core_web_sm")

In [4]:
import vectorian
from vectorian.embeddings import CompressedFastTextVectors, PretrainedGloVe

fasttext = CompressedFastTextVectors("../models/fasttext-en-mini.kv")
glove = PretrainedGloVe("6B", 50)

from vectorian.importers import NovelImporter
from vectorian.session import LabSession
from vectorian.embeddings import SpacyTransformerEmbedding

# contextual embeddings
#im = NovelImporter(nlp, embeddings=[SpacyTransformerEmbedding(nlp)])

im = NovelImporter(nlp)
doc = im("/Users/arbeit/Wise Children.txt")

# note: you may also specify StackedEmbedding here.
session = LabSession(
    [doc],
    [fasttext, glove],
    "default")

Matching checksum for /Users/arbeit/Projects/vectorian-2021/vectorian/core/cpp/core.cpp --> not compiling


Importing /Users/arbeit/Wise Children.txt: 100%|██████████| 369/369 [00:02<00:00, 158.35it/s]
Opening glove-6B-50: 100%|██████████
4054it [00:00, 35450.69it/s]


In [5]:
session.interact(nlp)